### Import Required Libraries and Set Up Environment Variables

In [59]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [60]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [61]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# URL, Query begins (?), Parameter name (), Equal sign (=), 
# field name ([@field:fieldname1]), query string operator (&), etc.
nyt_query_url = f"{url}&api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
# print(nyt_query_url)

In [62]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
        # create query with a page number
    # API results show 10 articles at a time
    nyt_query_url_page = f"{nyt_query_url}&page={page}"
    nyt_query_url_page
    # Make a "GET" request and retrieve the JSON
    # is the 2nd line necessary or should I combine these? is this even correct?
    response = requests.get(nyt_query_url_page)
    reviews = response.json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            # print(review)
        # Print the page that was just retrieved
        print(f"Checked page {page}")
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results from page {page}")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [63]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_five_results = reviews_list[:5]
json_results = json.dumps(first_five_results, indent=4)
print(json_results)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [64]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = pd.json_normalize(reviews_list)
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [65]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# \u2018 = `
# \u2019 = `
# End string should include " Review" to avoid cutting title early
reviews_list_df["title"] = reviews_list_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1: st.find("\u2019 Review")])
reviews_list_df
# head_line=reviews_list_df["headline.main"]
# head_line

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [66]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df["keywords"] = reviews_list_df["keywords"].apply(extract_keywords)
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [67]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
# title=reviews_list_df["title"]
# title
title_list=list(reviews_list_df["title"])
title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [68]:
# Prepare The Movie Database query
url2 = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [77]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
counter = 0

# Loop through the titles
for title in title_list[:]:
    # Check if we need to sleep before making a request               
    # Add 1 to the request counter
    counter += 1
    if counter % 50 == 0:
        time.sleep(1)        
    
    # Perform a "GET" request for The Movie Database
    # Query URL
    tmdb_query_url = f'{url2}{title}{tmdb_key_string}'
    movie_response = requests.get(tmdb_query_url)
    movie_data = movie_response.json()    
        
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:

        # Get movie id
        movie_id = movie_data["results"][0]["id"]
        # print(movie_id)

        # Make a request for a the full movie details     
        full_movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}'
        # print(full_movie_url)

        # Execute "GET" request with url
        movie_response = requests.get(tmdb_query_url)
        movie_data = movie_response.json()
        full_movie_response = requests.get(full_movie_url)
        full_movie_results = full_movie_response.json()
        # print(json.dumps(full_movie_results, indent=4))
        
        # Extract the genre names into a list
        genre_names = full_movie_results["genres"][0]["name"]

        # Extract the spoken_languages' English name into a list
        spoken_languages = full_movie_results["spoken_languages"][0]["english_name"]

        # Extract the production_countries' name into a list
        production_countries = full_movie_results["production_countries"][0]["name"]

        # Add the relevant data to a dictionary and
        # need to add title, original_title, budget, original_language, 
        # homepage, overview, popularity, runtime, revenue, release_date, vote_average
        # genres, spoken_language, production_countries
        data_dictionary = {
            'title': title,
            'original_title': full_movie_results['original_title'],
            'budget': full_movie_results['budget'],
            'genre': genre_names,
            'language': full_movie_results['original_language'],
            'spoken_languages': spoken_languages,
            'homepage': full_movie_results['homepage'],
            'overview': full_movie_results['overview'],
            'popularity': full_movie_results['popularity'],
            'runtime' : full_movie_results['runtime'],
            'revenue' : full_movie_results['revenue'],
            'release_date': full_movie_results['release_date'],
            'vote_average' : full_movie_results['vote_average'],
            'vote_count' : full_movie_results['vote_count'],
            'production_countries': production_countries

        }
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(data_dictionary)
    

        # Print out the title that was found
        print(f'Found {title}')


    except IndexError:
        print(f'{title} not found.')


Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
In a New York Minute not found.
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and D

In [78]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": "Drama",
        "language": "es",
        "spoken_languages": "Spanish",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.989,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "production_countries": "Argentina"
    },
    {
        "title": "You Can Live Forever",
        "original_title": "You Can Live Forever",
        "budget": 0,
        "genre": "Drama",
 

In [79]:
# Convert the results to a DataFrame
tmdb_movies_list_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_list_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,Drama,es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.989,102,0,2021-10-07,3.000,4,Argentina
1,You Can Live Forever,You Can Live Forever,0,Drama,en,English,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.997,96,15055,2023-03-24,6.622,37,Canada
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,Romance,en,English,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",16.495,96,0,2023-04-21,6.287,162,United States of America
3,Other People’s Children,Les Enfants des autres,0,Drama,fr,French,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.464,104,84178,2022-09-21,6.818,190,France
4,One True Loves,One True Loves,0,Romance,en,English,,Emma and Jesse are living the perfect life tog...,9.862,100,37820,2023-04-07,6.527,75,Czech Republic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,The Last Face,The Last Face,0,Drama,en,English,,"Miguel, a heroic Spanish doctor, puts himself ...",12.385,130,0,2017-01-11,5.600,193,United States of America
147,Lost in Paris,Paris pieds nus,0,Comedy,fr,French,,Fiona visits Paris for the first time to assis...,7.667,83,0,2017-01-14,6.022,114,Belgium
148,The Other Half,The Other Half,0,Drama,en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.050,103,0,2016-12-02,6.300,24,Canada
149,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,Romance,en,Turkish,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",14.144,111,413844,2017-03-10,6.164,247,Turkey


### Merge and Clean the Data for Export

In [80]:
# Merge the New York Times reviews and TMDB DataFrames on title
# reviews_list_df.set_index('title', inplace=True)
# tmdb_movies_list_df.set_index('title', inplace=True)
merge_df = pd.merge(tmdb_movies_list_df, reviews_list_df, on="title")
merge_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,Drama,es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.989,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,Drama,en,English,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.997,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,Romance,en,English,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",16.495,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,Other People’s Children,Les Enfants des autres,0,Drama,fr,French,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.464,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
4,One True Loves,One True Loves,0,Romance,en,English,,Emma and Jesse are living the perfect life tog...,9.862,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,The Last Face,The Last Face,0,Drama,en,English,,"Miguel, a heroic Spanish doctor, puts himself ...",12.385,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
147,Lost in Paris,Paris pieds nus,0,Comedy,fr,French,,Fiona visits Paris for the first time to assis...,7.667,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
148,The Other Half,The Other Half,0,Drama,en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.050,103,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
149,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,Romance,en,Turkish,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",14.144,111,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None


In [81]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["]", "]", "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
      merge_df[col] = merge_df[col].astype(str)

    # Loop through characters to remove
      for char in characters_to_remove:
            merge_df[col] = merge_df[col].str.replace(char, " " ) 

# Display the fixed DataFrame
merge_df

KeyError: 'genres'

In [82]:
# Drop "byline.person" column
# axis = 0 used for rows
merge_df = merge_df.drop('byline.person', axis=1)
merge_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,Drama,es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.989,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,Drama,en,English,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.997,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,Romance,en,English,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",16.495,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
3,Other People’s Children,Les Enfants des autres,0,Drama,fr,French,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.464,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
4,One True Loves,One True Loves,0,Romance,en,English,,Emma and Jesse are living the perfect life tog...,9.862,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,The Last Face,The Last Face,0,Drama,en,English,,"Miguel, a heroic Spanish doctor, puts himself ...",12.385,130,...,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
147,Lost in Paris,Paris pieds nus,0,Comedy,fr,French,,Fiona visits Paris for the first time to assis...,7.667,83,...,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
148,The Other Half,The Other Half,0,Drama,en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.050,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
149,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,Romance,en,Turkish,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",14.144,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None


In [83]:
# Delete duplicate rows and reset index
merge_df = merge_df.drop_duplicates()
merge_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,Drama,es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.989,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,Drama,en,English,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.997,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,Romance,en,English,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",16.495,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
3,Other People’s Children,Les Enfants des autres,0,Drama,fr,French,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.464,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
4,One True Loves,One True Loves,0,Romance,en,English,,Emma and Jesse are living the perfect life tog...,9.862,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,The Last Face,The Last Face,0,Drama,en,English,,"Miguel, a heroic Spanish doctor, puts himself ...",12.385,130,...,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
147,Lost in Paris,Paris pieds nus,0,Comedy,fr,French,,Fiona visits Paris for the first time to assis...,7.667,83,...,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
148,The Other Half,The Other Half,0,Drama,en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.050,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
149,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,Romance,en,Turkish,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",14.144,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None


In [ ]:
# Export data to CSV without the index
merge_df.to_csv('merge.csv', index=False)